In [2]:
%matplotlib inline
import numpy as np
import pandas as pd
from tqdm import tqdm_notebook as tqdm
from sklearn.externals import joblib
import seaborn as sns

In [4]:
dst = pd.read_csv('./data/application_train.csv')

307511
307511
307511
307511
307511
307511
307511
307511
307511
307499
307233
306219
307511
307511
307511
307511
307511
307511
307511
307511
307511
104582
307511
307511
307511
307511
307511
307511
211120
307509
307511
307511
307511
307511
307511
307511
307511
307511
307511
307511
307511
134133
306851
246546
151450
127568
157504
103023
92646
143620
152683
154491
98869
124921
97312
153161
93997
137829
151450
127568
157504
103023
92646
143620
152683
154491
98869
124921
97312
153161
93997
137829
151450
127568
157504
103023
92646
143620
152683
154491
98869
124921
97312
153161
93997
137829
97216
153214
159080
151170
161756
306490
306490
306490
306490
307510
307511
307511
307511
307511
307511
307511
307511
307511
307511
307511
307511
307511
307511
307511
307511
307511
307511
307511
307511
307511
265992
265992
265992
265992
265992
265992


In [25]:
dst.head()

,SK_ID_CURR,TARGET,NAME_CONTRACT_TYPE,CODE_GENDER,FLAG_OWN_CAR,FLAG_OWN_REALTY,CNT_CHILDREN,AMT_INCOME_TOTAL,AMT_CREDIT,AMT_ANNUITY,...,FLAG_DOCUMENT_18,FLAG_DOCUMENT_19,FLAG_DOCUMENT_20,FLAG_DOCUMENT_21,AMT_REQ_CREDIT_BUREAU_HOUR,AMT_REQ_CREDIT_BUREAU_DAY,AMT_REQ_CREDIT_BUREAU_WEEK,AMT_REQ_CREDIT_BUREAU_MON,AMT_REQ_CREDIT_BUREAU_QRT,AMT_REQ_CREDIT_BUREAU_YEAR
0,100002,1,Cash loans,M,N,Y,0,202500.0,406597.5,24700.5,...,0,0,0,0,0.0,0.0,0.0,0.0,0.0,1.0
1,100003,0,Cash loans,F,N,N,0,270000.0,1293502.5,35698.5,...,0,0,0,0,0.0,0.0,0.0,0.0,0.0,0.0
2,100004,0,Revolving loans,M,Y,Y,0,67500.0,135000.0,6750.0,...,0,0,0,0,0.0,0.0,0.0,0.0,0.0,0.0
3,100006,0,Cash loans,F,N,Y,0,135000.0,312682.5,29686.5,...,0,0,0,0,NaN,NaN,NaN,NaN,NaN,NaN
4,100007,0,Cash loans,M,N,Y,0,121500.0,513000.0,21865.5,...,0,0,0,0,0.0,0.0,0.0,0.0,0.0,0.0


In [8]:
dsccb = pd.read_csv('./data/credit_card_balance.csv')
dsccb.head()

,SK_ID_PREV,SK_ID_CURR,MONTHS_BALANCE,AMT_BALANCE,AMT_CREDIT_LIMIT_ACTUAL,AMT_DRAWINGS_ATM_CURRENT,AMT_DRAWINGS_CURRENT,AMT_DRAWINGS_OTHER_CURRENT,AMT_DRAWINGS_POS_CURRENT,AMT_INST_MIN_REGULARITY,...,AMT_RECIVABLE,AMT_TOTAL_RECEIVABLE,CNT_DRAWINGS_ATM_CURRENT,CNT_DRAWINGS_CURRENT,CNT_DRAWINGS_OTHER_CURRENT,CNT_DRAWINGS_POS_CURRENT,CNT_INSTALMENT_MATURE_CUM,NAME_CONTRACT_STATUS,SK_DPD,SK_DPD_DEF
0,2562384,378907,-6,56.970,135000,0.0,877.5,0.0,877.5,1700.325,...,0.000,0.000,0.0,1,0.0,1.0,35.0,Active,0,0
1,2582071,363914,-1,63975.555,45000,2250.0,2250.0,0.0,0.0,2250.000,...,64875.555,64875.555,1.0,1,0.0,0.0,69.0,Active,0,0
2,1740877,371185,-7,31815.225,450000,0.0,0.0,0.0,0.0,2250.000,...,31460.085,31460.085,0.0,0,0.0,0.0,30.0,Active,0,0
3,1389973,337855,-4,236572.110,225000,2250.0,2250.0,0.0,0.0,11795.760,...,233048.970,233048.970,1.0,1,0.0,0.0,10.0,Active,0,0
4,1891521,126868,-1,453919.455,450000,0.0,11547.0,0.0,11547.0,22924.890,...,453919.455,453919.455,0.0,1,0.0,1.0,101.0,Active,0,0


In [9]:
dsccb_g = dsccb.groupby('SK_ID_CURR').mean()

In [11]:
X_with_cc = pd.merge(dst, dsccb_g.reset_index(), how='left', on=['SK_ID_CURR'])
X_with_cc['AMT_BALANCE_LIMIT_RATIO'] = X_with_cc['AMT_BALANCE'] / X_with_cc['AMT_CREDIT_LIMIT_ACTUAL']

# result = pd.concat(frames, join='outer', keys="SK_ID_CURR")
# result.head()
credit_card_balance_columns = ['AMT_BALANCE_LIMIT_RATIO', 'MONTHS_BALANCE','AMT_BALANCE','AMT_CREDIT_LIMIT_ACTUAL','AMT_DRAWINGS_ATM_CURRENT','AMT_DRAWINGS_CURRENT','AMT_DRAWINGS_OTHER_CURRENT','AMT_DRAWINGS_POS_CURRENT','AMT_INST_MIN_REGULARITY','AMT_PAYMENT_CURRENT','AMT_PAYMENT_TOTAL_CURRENT','AMT_RECEIVABLE_PRINCIPAL','AMT_RECIVABLE','AMT_TOTAL_RECEIVABLE','CNT_DRAWINGS_ATM_CURRENT','CNT_DRAWINGS_CURRENT','CNT_DRAWINGS_OTHER_CURRENT','CNT_DRAWINGS_POS_CURRENT','CNT_INSTALMENT_MATURE_CUM']
X_cc = X_with_cc[credit_card_balance_columns + ['TARGET']]
X_cc_corr = abs(X_cc.corr())

In [12]:
X_cc.count()
# X_cc_corr.sort_values('TARGET', ascending=False)['TARGET']

AMT_BALANCE_LIMIT_RATIO        86163
MONTHS_BALANCE                 86905
AMT_BALANCE                    86905
AMT_CREDIT_LIMIT_ACTUAL        86905
AMT_DRAWINGS_ATM_CURRENT       61140
AMT_DRAWINGS_CURRENT           86905
AMT_DRAWINGS_OTHER_CURRENT     61140
AMT_DRAWINGS_POS_CURRENT       61140
AMT_INST_MIN_REGULARITY        86905
AMT_PAYMENT_CURRENT            61060
AMT_PAYMENT_TOTAL_CURRENT      86905
AMT_RECEIVABLE_PRINCIPAL       86905
AMT_RECIVABLE                  86905
AMT_TOTAL_RECEIVABLE           86905
CNT_DRAWINGS_ATM_CURRENT       61140
CNT_DRAWINGS_CURRENT           86905
CNT_DRAWINGS_OTHER_CURRENT     61140
CNT_DRAWINGS_POS_CURRENT       61140
CNT_INSTALMENT_MATURE_CUM      86905
TARGET                        307511
dtype: int64

In [17]:
dsbb = pd.read_csv('./data/bureau.csv')


SK_ID_CURR                1716428
SK_ID_BUREAU              1716428
CREDIT_ACTIVE             1716428
CREDIT_CURRENCY           1716428
DAYS_CREDIT               1716428
CREDIT_DAY_OVERDUE        1716428
DAYS_CREDIT_ENDDATE       1610875
DAYS_ENDDATE_FACT         1082775
AMT_CREDIT_MAX_OVERDUE     591940
CNT_CREDIT_PROLONG        1716428
AMT_CREDIT_SUM            1716415
AMT_CREDIT_SUM_DEBT       1458759
AMT_CREDIT_SUM_LIMIT      1124648
AMT_CREDIT_SUM_OVERDUE    1716428
CREDIT_TYPE               1716428
DAYS_CREDIT_UPDATE        1716428
AMT_ANNUITY                489637
dtype: int64

In [29]:
dsbb_g = dsbb.groupby('SK_ID_CURR').mean()
dsbb_g = dsbb_g.reset_index()

In [35]:
X_with_bb = pd.merge(dst, dsbb_g, how='left', on=['SK_ID_CURR'])
bb_columns = ['DAYS_CREDIT','CREDIT_DAY_OVERDUE','DAYS_CREDIT_ENDDATE','DAYS_ENDDATE_FACT','AMT_CREDIT_MAX_OVERDUE','CNT_CREDIT_PROLONG','AMT_CREDIT_SUM','AMT_CREDIT_SUM_DEBT','AMT_CREDIT_SUM_LIMIT','AMT_CREDIT_SUM_OVERDUE','DAYS_CREDIT_UPDATE']
X_bb = X_with_bb[bb_columns + ['TARGET']]
X_bb_corr = abs(X_bb.corr())
X_bb_corr.sort_values('TARGET', ascending=False)['TARGET']

TARGET                    1.000000
DAYS_CREDIT               0.089729
DAYS_CREDIT_UPDATE        0.068927
DAYS_ENDDATE_FACT         0.053200
DAYS_CREDIT_ENDDATE       0.046983
AMT_CREDIT_SUM            0.019957
AMT_CREDIT_SUM_LIMIT      0.011446
CREDIT_DAY_OVERDUE        0.008118
AMT_CREDIT_SUM_OVERDUE    0.007150
CNT_CREDIT_PROLONG        0.003031
AMT_CREDIT_MAX_OVERDUE    0.002435
AMT_CREDIT_SUM_DEBT       0.000637
Name: TARGET, dtype: float64

In [37]:
X_with_bb.count()

SK_ID_CURR                    307511
TARGET                        307511
NAME_CONTRACT_TYPE            307511
CODE_GENDER                   307511
FLAG_OWN_CAR                  307511
FLAG_OWN_REALTY               307511
CNT_CHILDREN                  307511
AMT_INCOME_TOTAL              307511
AMT_CREDIT                    307511
AMT_ANNUITY_x                 307499
AMT_GOODS_PRICE               307233
NAME_TYPE_SUITE               306219
NAME_INCOME_TYPE              307511
NAME_EDUCATION_TYPE           307511
NAME_FAMILY_STATUS            307511
NAME_HOUSING_TYPE             307511
REGION_POPULATION_RELATIVE    307511
DAYS_BIRTH                    307511
DAYS_EMPLOYED                 307511
DAYS_REGISTRATION             307511
DAYS_ID_PUBLISH               307511
OWN_CAR_AGE                   104582
FLAG_MOBIL                    307511
FLAG_EMP_PHONE                307511
FLAG_WORK_PHONE               307511
FLAG_CONT_MOBILE              307511
FLAG_PHONE                    307511
F

In [35]:
dspc = pd.read_csv('./data/POS_CASH_balance.csv')
dspc.head()

,SK_ID_PREV,SK_ID_CURR,MONTHS_BALANCE,CNT_INSTALMENT,CNT_INSTALMENT_FUTURE,NAME_CONTRACT_STATUS,SK_DPD,SK_DPD_DEF
0,1803195,182943,-31,48.0,45.0,Active,0,0
1,1715348,367990,-33,36.0,35.0,Active,0,0
2,1784872,397406,-32,12.0,9.0,Active,0,0
3,1903291,269225,-35,48.0,42.0,Active,0,0
4,2341044,334279,-35,36.0,35.0,Active,0,0


In [36]:
X_with_pc = pd.merge(dst, dspc, how='left', on=['SK_ID_CURR'])
pc_columns = ['MONTHS_BALANCE','CNT_INSTALMENT','CNT_INSTALMENT_FUTURE','NAME_CONTRACT_STATUS','SK_DPD', 'SK_DPD_DEF']
X_pc = X_with_pc[pc_columns + ['TARGET']]
X_pc_corr = abs(X_pc.corr())
X_pc_corr.sort_values('TARGET', ascending=False)['TARGET']

TARGET                   1.000000
CNT_INSTALMENT_FUTURE    0.021972
MONTHS_BALANCE           0.020147
CNT_INSTALMENT           0.018506
SK_DPD                   0.009866
SK_DPD_DEF               0.008594
Name: TARGET, dtype: float64

In [4]:
dsip = pd.read_csv('./data/installments_payments.csv')
dsip.head()

,SK_ID_PREV,SK_ID_CURR,NUM_INSTALMENT_VERSION,NUM_INSTALMENT_NUMBER,DAYS_INSTALMENT,DAYS_ENTRY_PAYMENT,AMT_INSTALMENT,AMT_PAYMENT
0,1054186,161674,1.0,6,-1180.0,-1187.0,6948.360,6948.360
1,1330831,151639,0.0,34,-2156.0,-2156.0,1716.525,1716.525
2,2085231,193053,2.0,1,-63.0,-63.0,25425.000,25425.000
3,2452527,199697,1.0,3,-2418.0,-2426.0,24350.130,24350.130
4,2714724,167756,1.0,2,-1383.0,-1366.0,2165.040,2160.585


In [5]:
X_with_ip = pd.merge(dst, dsip, how='left', on=['SK_ID_CURR'])
ip_columns = ['NUM_INSTALMENT_VERSION','NUM_INSTALMENT_NUMBER','DAYS_INSTALMENT','DAYS_ENTRY_PAYMENT','AMT_INSTALMENT', 'AMT_PAYMENT']
X_ip = X_with_ip[ip_columns + ['TARGET']]
X_ip_corr = abs(X_ip.corr())
X_ip_corr.sort_values('TARGET', ascending=False)['TARGET']

TARGET                    1.000000
DAYS_ENTRY_PAYMENT        0.035122
DAYS_INSTALMENT           0.034974
NUM_INSTALMENT_NUMBER     0.016190
NUM_INSTALMENT_VERSION    0.009896
AMT_PAYMENT               0.003623
AMT_INSTALMENT            0.001498
Name: TARGET, dtype: float64